In [1]:
import ee
ee.Initialize()
import eeSAR
from eeSAR.s1 import s1_collection, s1_timescan
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
stations = gpd.read_file('/home/vollrath/Indonesia_sm/stations_pix_corr.shp')
gdf = gpd.read_file('/home/vollrath/Indonesia_sm/ground_truth_sm.gpkg')

In [3]:
grouped = gdf.groupby('station').first().reset_index()
grouped.head()

,station,index,location,province,lon,lat,GWL_max,GWL_min,GWL_rata,SM_max,SM_min,SM_rata,Total,geometry
0,BRG_150503_01,06/07/2019,BETUNG,JAMBI,103.928286,-1.545325,-3.040,-3.040,-3.040,62.413,61.607,62.024,0.00,POINT (103.92829 -1.54533)
1,BRG_150604_01,31/07/2019,PEMATANG BULUH,JAMBI,103.371518,-1.081556,-3.523,-3.523,-3.523,58.500,204.500,203.738,0.00,POINT (103.37152 -1.08156)
2,BRG_150604_02,01/08/2019,SUNGAI TERAP,JAMBI,103.427238,-0.953954,27.860,-3.840,-3.537,22.900,-1.700,200.368,0.10,POINT (103.42724 -0.95395)
3,BRG_150611_01,31/10/2018,Bram Itam Kanan,JAMBI,103.321105,-0.908060,-2.237,-2.300,-2.280,7.900,0.000,17.640,0.10,POINT (103.32111 -0.90806)
4,BRG_150611_02,30/10/2018,Bram Itam Raya,JAMBI,103.354661,-0.921704,-1.888,-2.037,-1.947,18.000,14.400,16.056,0.05,POINT (103.35466 -0.92170)


In [4]:
def extract_ts(i, row):
    
    # get station name
    station = row['station']
    
    # create EE point geometry
    point = ee.Geometry.Point(row['lon'], row['lat'])
    
    # create collection for time-series
    s1_preproc = s1_collection.create(
        region=point.buffer(1000), 
        start_date='2014-08-01',
        end_date='2020-10-31', 
        add_ND_ratio=False,
        add_ratio=False,
        speckle_filter='QUEGAN',
        radiometric_correction='TERRAIN',
        slope_correction_dict={'model': 'surface', 'dem': 'USGS/SRTMGL1_003', 'buffer': 50}, #'CGIAR/SRTM90_V4'
        db=True
    )
    
    # get EE's scene items and S1 scene_ids
    scenes = s1_preproc.getInfo().get('features')
    images = [item.get('properties')['system:index'] for item in scenes]
    
    
    # extract value for each image in time-series collection
    ts = []
    for scene_id in images:
        image = s1_preproc.filter(ee.Filter.eq('system:index', scene_id)).first()
        proj = image.select('VV').projection()#.getInfo()
        # get actual data
        image_red = image.reduceRegion(
            reducer=ee.Reducer.toList(),\
            geometry=point,\
            maxPixels=1e13,\
            crs=proj,\
            scale=100
        );
        
        # get data from EE
        data_dict = image_red.select(['VV', 'VH', 'angle', 'LIA']).getInfo()
        props = image.getInfo()['properties']
        scene_id = props['system:index']
        orbit = props['orbitProperties_pass']
        orbit = 1 if orbit == 'ASCENDING' else 2
        track = props['relativeOrbitNumber_start']
        
        # and add it those as tuple ot the ts list
        ts.append((scene_id, scene_id[17:25], track, orbit, data_dict))
    
    # turn the ts list into a dataframe
    d = {}
    d[station] = ts
    df = pd.DataFrame.from_dict(d)

    # split the data within the data frame to get actual structured data
    d2 = {}
    for j, ts in df.iterrows():

        values = ts[0]
        ts['scene_id'] = values[0]
        ts['date'] = values[1]
        ts['track'] = values[2]
        ts['orbit'] = values[3]
        ts['VV'] = values[4]['VV'][0] if values[4]['VV'] else np.nan
        ts['VH'] = values[4]['VH'][0] if values[4]['VH'] else np.nan
        ts['angle'] = values[4]['angle'][0] if values[4]['angle'] else np.nan
        ts['LIA'] = values[4]['LIA'][0] if values[4]['LIA'] else np.nan

        d2[j] = ts

    # re-create a dataframe and delete the tupled data
    ts_df = pd.DataFrame.from_dict(d2, orient='index')
    del ts_df[station]
    
    # and finally write to output file
    ts_df.to_pickle(f'/home/vollrath/Indonesia_sm/s1_timeseries/{i}.{station}.s1.ts.pickle')

    return ts
    

In [5]:
from multiprocessing import Process

ts = {}
for i, row in grouped.iterrows():
    
    if i != 45:
        continue
    
    ec = None
    while ec is None != 0:
        p1 = Process(target=extract_ts, args=(i, row, ), name='Process')
        p1.start()
        p1.join(timeout=500)
        p1.terminate()
        ec = p1.exitcode
    
    
